In [2]:
import re, string, json, time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from joblib import dump, load
from pathlib import Path
from flask import Flask, jsonify, request

from progress.bar import *
from progress.spinner import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import Word

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brogan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
#lowercase, remove([^\w\s]), remove(\d), remove(stopwords), Word(word).lemmatize() for word in news, remove(r'http\S+')
news = "rocks jasdf sodHHHas 1299 .^\n"
news = news.lower()
news = news.replace('[^\w\s]', '')
news = news.replace(r'\d', '')

stop_words = set(stopwords.words('english'))  # get english stopwords
punctuation = list(string.punctuation)  # get punc
stop_words.update(punctuation)
news = ' '.join([word for word in news.split() if word not in stop_words])
news = ' '.join([Word(word).lemmatize() for word in news.split() ])
news = news.replace(r'http\S+', '')

print(news)

rock jasdf sodHHHas 1299 .^


In [14]:
df_test = pd.read_csv("data/orig/WELFake_Dataset.csv")
df_test.drop_duplicates(inplace = True)
df_test.dropna(inplace = True)
df_test['text'] = df_test['text'] + " " + df_test['title']
df_test.drop(['label', 'title'], axis=1, inplace=True)
df_test

,Unnamed: 0,text
0,0,No comment is expected from Barack Obama Membe...
2,2,"Now, most of the demonstrators gathered last ..."
3,3,A dozen politically active pastors came here f...
4,4,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,5,All we can say on this one is it s about time ...
...,...,...
72129,72129,WASHINGTON (Reuters) - Hackers believed to be ...
72130,72130,"You know, because in fantasyland Republicans n..."
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...
72132,72132,MEXICO CITY (Reuters) - Donald Trump’s combati...


In [15]:
from sklearn.utils import shuffle

In [16]:
df_test = pd.read_csv("data/orig/FakeNewsNet.csv")
df_test.drop_duplicates(inplace = True)
df_test.dropna(inplace = True)
#df_test['text'] = df_test['text'] + " " + df_test['title'] #80.76% with this, same without
df_test.drop(['title'], axis=1, inplace=True)
df_test = shuffle(df_test)

In [17]:
df_test

,news_url,source_domain,tweet_num,real
18965,https://www.whimn.com.au/look/beauty/why-liz-h...,www.whimn.com.au,0,1
17400,www.today.com/popculture/blake-shelton-remembe...,www.today.com,6,0
17691,https://www.businessinsider.com/sarah-hyland-h...,www.businessinsider.com,72,1
7451,http://nrtonline.info/breaking-nancy-pelosi-to...,nrtonline.info,109,0
14754,https://pagesix.com/2018/08/25/angelina-jolie-...,pagesix.com,40,1
...,...,...,...,...
7518,https://www.thehollywoodgossip.com/2018/01/khl...,www.thehollywoodgossip.com,1,1
12140,https://www.romper.com/p/jill-duggar-might-hav...,www.romper.com,23,1
499,https://www.seventeen.com/celebrity/celebrity-...,www.seventeen.com,64,1
9469,https://www.usatoday.com/story/life/entertaint...,www.usatoday.com,74,1
